In [1]:
"""
9/18 using UNET muliticlass image segmentation 
https://github.com/bnsreenu/python_for_image_processing_APEER/blob/master/tutorial120_applying_trained_unet_model_to_large_images.ipynb
version for rtx3090
NEED TO take white 10x10 px space into account

9/22 - removed white space, optimized onehot encoding procedure

9/28 - Trying 512x512 version
"""
import os
import glob
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import sys
print(sys.version)
from keras import layers
from tensorflow.keras.preprocessing.image import load_img
import cv2
import numpy as np


gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))
print(gpus[0])

3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
Num GPUs Available:  1
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [2]:
tf.test.is_built_with_cuda()
#print(os.getcwd())

True

In [3]:
#os.chdir("..")
#os.chdir(os. getcwd() + "\\src")
#from c import ROOT_DIR

ROOT_DIR = 'c:\\Users\\endle\\Desktop\\mseisML\\src'
os.chdir(ROOT_DIR)


In [4]:
"""
Add Drive for dataset
"""
# local

ROOT_DIR = 'c:\\Users\\endle\\Desktop\\mseisML'

mask_path = ROOT_DIR + "\\MLdata_unconditioned_sept19\\masks\\"
image_path = ROOT_DIR + "\\MLdata_unconditioned_sept19\\das\\"

""

''

In [5]:
#Resizing images, if needed
SIZE_X = 512 #128 
SIZE_Y = 512 #128
n_classes=3 #Number of classes for segmentation
num_images = 2612  #Total 1600 available


In [6]:
"""
methods to one-hot encode data
"""

# import math

# no_data = str([0,0,0])
# p_wave = str([100,0,0])
# s_wave = str([0,0,100])

# labels = {no_data:0, p_wave:1, s_wave:2}

# COLORS = ((0,0,0), (100,0,0), (0,0,100))

# def closest_color(rgb, COLORS):

#     """ find closest color from dict"""
#     r, g, b = rgb
#     color_diffs = []
#     for color in COLORS:
#         cr, cg, cb = color
#         color_diff = math.sqrt((r - cr)**2 + (g - cg)**2 + (b - cb)**2)
#         color_diffs.append((color_diff, color))

#     return min(color_diffs)[1]

# def label_pixel(path, COLORS, labels):
    
#     """
#     create mask with label values as pixel
#     this make take some time...

#     """
    
#     #x = img_size

#     #img = load_img(path, target_size=self.img_size, color_mode="rgb")
#     img = cv2.imread(path)
#     #img = cv2.resize(img, img_size)

#     x = np.shape(img)

#     #print('shape of x, ', x)
#     values = [str(list(img[i,j])) for i in range(x[0]) for j in range(x[1])]
#     mask=list([0]*x[0]*x[1])


#     for i, value in enumerate(values):

#         value = value.replace('[','').replace(']','').split(',')
#         value = (int(value[0]), int(value[1]), int(value[2]))   
#         value = closest_color(value, COLORS)
#         value = list(value)   
#         value = '[' + str(value[0]) + ', ' + str(value[1]) + ', ' + str(value[2]) + ']'   
#         mask[i]=labels[value]

#     #print('shape of mask before reshape', np.shape(mask))
#     #print('shape of mask after reshape', np.shape(np.asarray(mask).reshape(x[0],x[1])))

#     return np.asarray(mask).reshape(x[0],x[1])

'\nmethods to one-hot encode data\n'

In [7]:
"""updated image routine"""

bordr = 11 # saved images have border from matplotlib that can't be removed before saving.

image_names = glob.glob(image_path + "*.png")
image_names.sort()
image_names_subset = image_names[0:num_images]
#images = [cv2.imread(image, 0) for image in image_names_subset]

images = []

for image in image_names_subset:

    
    x = cv2.imread(image)
    x_n = x[0:x.shape[1]-bordr, bordr:x.shape[0]][:]
    x_f = cv2.resize(x_n, (SIZE_X,SIZE_Y), 3, interpolation=cv2.INTER_NEAREST)


    # # for greyscale inputs
    # x = cv2.imread(image, 0)
    # x_n = x[0:x.shape[1]-bordr, bordr:x.shape[0]]
    # x_f = cv2.resize(x_n, (SIZE_X,SIZE_Y), interpolation=cv2.INTER_NEAREST)

    images.append(x_f)

image_dataset = np.array(images)
#image_dataset = np.expand_dims(image_dataset, axis = 3) #use for greyscale

print('shape of image_dataset', np.shape(image_dataset))


shape of image_dataset (2612, 512, 512, 3)


In [8]:
"""updated mask routine"""

mask_names = glob.glob(mask_path + "*.png")
mask_names.sort()
mask_names_subset = mask_names[0:num_images]

masks = []

for mask in mask_names_subset:

    #do
    # Load the image and obtain tensor with one-hot values

    image = cv2.imread(mask)

    # collapse on color

    one_hot = np.asarray(np.round((1/255)*image[:,:,0])  + 2*np.round((1/255)*image[:,:,2]), dtype=np.int32)

    # remove white border

    one_hot = one_hot[0:one_hot.shape[1]-bordr, bordr:one_hot.shape[0]]

    # resize

    one_hot_c = cv2.resize(one_hot, (SIZE_X,SIZE_Y), interpolation=cv2.INTER_NEAREST)
    
    masks.append(one_hot_c)


print('shape of masks', np.shape(masks))

mask_dataset = np.array(masks) # should be: shape of masks (2612, 128, 128)


shape of masks (2612, 512, 512)


In [9]:
### old routine, it is slow.. delete later

# # mask_names = glob.glob(mask_path + "*.png")
# # mask_names.sort()
# # mask_names_subset = mask_names[0:num_images]
# # masks = [cv2.imread(mask, 0) for mask in mask_names_subset]
# # mask_dataset = np.array(masks)

# mask_names = glob.glob(mask_path + "*.png")
# mask_names.sort()
# mask_names_subset = mask_names[0:num_images]

# #print('subset, ', mask_names_subset) # prints out

# #convert masks from 3 channel into one hot encoding per pixel
# #masks = [cv2.imread(mask, 0) for mask in mask_names_subset]

# masks = []

# for mask in mask_names_subset:
#     y = label_pixel(mask, COLORS, labels)
#     #print('shape of y:', np.shape(y))
#     #plt.imshow(y)
#     #cax = plt.axes([0.85, 0.1, 0.075, 0.8])
#     #plt.colorbar(cax=cax)
#     #plt.show()


#     #fig, axs = plt.subplots(1,2, figsize=(9, 3), sharey=True)
#     #axs[0].imshow(y)
    
#     y_n = y[0:y.shape[1]-10, 10:y.shape[0]]
    
#     y_f = cv2.resize(y_n, (SIZE_X,SIZE_Y), interpolation=cv2.INTER_NEAREST)


#     masks.append(y_f)

# print('shape of masks', np.shape(masks))

# mask_dataset = np.array(masks)


In [10]:
print("Image data shape is: ", image_dataset.shape)
print("Mask data shape is: ", mask_dataset.shape)
print("Max pixel value in image is: ", image_dataset.max())
print("Labels in the mask are : ", np.unique(mask_dataset))

Image data shape is:  (2612, 512, 512, 3)
Mask data shape is:  (2612, 512, 512)
Max pixel value in image is:  255
Labels in the mask are :  [0 1 2]


In [11]:
#Encode labels to 0, 1, 2, 3, ... but multi dim array so need to flatten, encode and reshape
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = mask_dataset.shape  
mask_dataset_reshaped = mask_dataset.reshape(-1,1)
mask_dataset_reshaped_encoded = labelencoder.fit_transform(mask_dataset_reshaped)
mask_dataset_encoded = mask_dataset_reshaped_encoded.reshape(n, h, w)

np.unique(mask_dataset_encoded)

c:\Users\endle\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 1, 2], dtype=int64)

In [12]:
mask_dataset_encoded = np.expand_dims(mask_dataset_encoded, axis = 3)
print(mask_dataset_encoded.shape)

(2612, 512, 512, 1)


In [13]:
#Normalize images
image_dataset = image_dataset /255.  #Can also normalize or scale using MinMax scaler

In [14]:
#Split training data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset_encoded, test_size = 0.2, random_state = 42)

In [15]:
from tensorflow.keras.utils import to_categorical
train_masks_cat = to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))

test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))

In [16]:
print(y_train_cat.shape)

(2089, 512, 512, 3)


In [17]:
"""
Updated UNET model, 512x512
"""
def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (3,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model

In [18]:
# # Building Unet by dividing encoder and decoder into blocks OLDER

# from keras.models import Model
# from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
# from keras.optimizers import Adam
# from keras.layers import Activation, MaxPool2D, Concatenate


# def conv_block(input, num_filters):
#     x = Conv2D(num_filters, 3, padding="same")(input)
#     x = BatchNormalization()(x)   #Not in the original network. 
#     x = Activation("relu")(x)

#     x = Conv2D(num_filters, 3, padding="same")(x)
#     x = BatchNormalization()(x)  #Not in the original network
#     x = Activation("relu")(x)

#     return x

# #Encoder block: Conv block followed by maxpooling


# def encoder_block(input, num_filters):
#     x = conv_block(input, num_filters)
#     p = MaxPool2D((2, 2))(x)
#     return x, p   

# #Decoder block
# #skip features gets input from encoder for concatenation

# def decoder_block(input, skip_features, num_filters):
#     x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
#     x = Concatenate()([x, skip_features])
#     x = conv_block(x, num_filters)
#     return x

# #Build Unet using the blocks
# def build_unet(input_shape, n_classes):
#     inputs = Input(input_shape)

#     s1, p1 = encoder_block(inputs, 64)
#     s2, p2 = encoder_block(p1, 128)
#     s3, p3 = encoder_block(p2, 256)
#     s4, p4 = encoder_block(p3, 512)

#     b1 = conv_block(p4, 1024) #Bridge

#     d1 = decoder_block(b1, s4, 512)
#     d2 = decoder_block(d1, s3, 256)
#     d3 = decoder_block(d2, s2, 128)
#     d4 = decoder_block(d3, s1, 64)

#     if n_classes == 1:  #Binary
#       activation = 'sigmoid'
#     else:
#       activation = 'softmax'

#     outputs = Conv2D(n_classes, 1, padding="same", activation=activation)(d4)  #Change the activation based on n_classes
#     print(activation)

#     model = Model(inputs, outputs, name="U-Net")
#     return model

In [19]:
"""just jaccard index for loss """

from keras import backend as K

def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)


def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)  # -1 ultiplied as we want to minimize this value as loss function

In [20]:
IMG_HEIGHT = X_train.shape[1]
IMG_WIDTH  = X_train.shape[2]
IMG_CHANNELS = X_train.shape[3]
input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
print('input shape, ', input_shape)

input shape,  (512, 512, 3)


In [21]:
#model = build_unet(input_shape, n_classes=n_classes)
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer='adam', loss=[jacard_coef_loss], metrics=[jacard_coef])
#model.summary()

img_size = (512, 512)
model = get_model(img_size, n_classes)
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [22]:
epochs = 25

print('shape of X_train', np.shape(X_train))
print('shape of y_train_cat', np.shape(y_train_cat))
print('shape of X_test', np.shape(X_test))
print('shape of y_test_cat', np.shape(y_test_cat))

history = model.fit(X_train, y_train_cat, 
                    batch_size = 16, 
                    verbose=1, 
                    epochs=epochs, 
                    validation_data=(X_test, y_test_cat), 
                    shuffle=False)

shape of X_train (2089, 512, 512, 3)
shape of y_train_cat (2089, 512, 512, 3)
shape of X_test (523, 512, 512, 3)
shape of y_test_cat (523, 512, 512, 3)
Epoch 1/25
 30/131 [=====>........................] - ETA: 1:00 - loss: 0.6633 - accuracy: 0.9269

ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/model/conv2d_7/Conv2D/Conv2DBackpropInput' defined at (most recent call last):
    File "c:\Users\endle\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\endle\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\endle\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\endle\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\endle\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "c:\Users\endle\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\endle\anaconda3\lib\asyncio\base_events.py", line 601, in ru

In [ ]:
#Save the model for future use

model_path = 'saved_model/UNET_512x512_25ep_sept28.hdf5'

model.save(model_path)

In [ ]:
###
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# include IOU
jacard = history.history['jacard_coef']
jacard_val = history.history['val_jacard_coef']

plt.plot(epochs, jacard, 'y', label='Training jacard')
plt.plot(epochs, jacard_val, 'r', label='Validation jacard')
plt.title('Training and validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('jacard')
plt.legend()
plt.show()

In [ ]:
#Load previously saved model
from keras.models import load_model
model = load_model(model_path, compile=False)

In [ ]:
def similarity_index(A,B,height,width):
    """https://www.delftstack.com/howto/python/opencv-compare-images/"""
    errorL2 = cv2.norm( A, B, cv2.NORM_L2 )
    return 1 - errorL2 / ( height * width )


In [ ]:
"""Look at results"""

for i in range(10):

    #do
    X_test_r  = np.reshape(X_test[0],(-1,128,128,1))
    print('shape of reshaped', np.shape(X_test_r))

    y_pred=model.predict(X_test_r)

    plt.figure(figsize=(12, 8))
    
    plt.subplot(141)
    plt.imshow(X_test[0])
    plt.title('Testing Image')

    plt.subplot(142)
    plt.imshow(y_test_cat[0])
    plt.title('Ground truth')

    plt.subplot(143)
    plt.imshow(y_pred[0])
    plt.title('Prediction')

    plt.subplot(144)
    y_pred_argmax=np.argmax(y_pred, axis=3)
    plt.imshow(y_pred_argmax[0])
    plt.title('Argmax Prediction')

    gt = y_test_cat[0]
    pd = y_pred_argmax[0]

    gt_g = cv2.cvtColor(gt, cv2.COLOR_BGR2GRAY)


    print('shapes', np.shape(gt), np.shape(pd))
    print('shapes', np.shape(gt_g), np.shape(pd))
    
    diff = gt_g - pd
    #s_index = similarity_index(gt_g, pd,IMG_HEIGHT, IMG_WIDTH)

    plt.show()

    plt.imshow(diff)

    print('sum of diff, ', np.sum(diff))
    plt.show()

    #print('sim index: ', s_index)



In [ ]:
y_pred_argmax=np.argmax(y_pred, axis=3)
y_pred_argmax.shape

print(y_pred_argmax)

In [ ]:
#Using built in keras function
from keras.metrics import MeanIoU
n_classes = 3
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(y_test[:,:,:,0], y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

In [ ]:
# #To calculate I0U for each class...
# values = np.array(IOU_keras.get_weights()).reshape(n_classes, n_classes)
# print(values)
# class1_IoU = values[0,0]/(values[0,0] + values[0,1] + values[0,2] + values[0,3] + values[1,0]+ values[2,0]+ values[3,0])
# class2_IoU = values[1,1]/(values[1,1] + values[1,0] + values[1,2] + values[1,3] + values[0,1]+ values[2,1]+ values[3,1])
# class3_IoU = values[2,2]/(values[2,2] + values[2,0] + values[2,1] + values[2,3] + values[0,2]+ values[1,2]+ values[3,2])
# #class4_IoU = values[3,3]/(values[3,3] + values[3,0] + values[3,1] + values[3,2] + values[0,3]+ values[1,3]+ values[2,3])

# print("IoU for class1 is: ", class1_IoU)
# print("IoU for class2 is: ", class2_IoU)
# print("IoU for class3 is: ", class3_IoU)
# #print("IoU for class4 is: ", class4_IoU)

In [ ]:
# run prediction on 50 images
import random

for i in range(50):

    test_img_number = random.randint(0, len(X_test)-1)
    test_img = X_test[test_img_number]
    ground_truth=y_test_cat[test_img_number]

    print('shape of test_img', np.shape(test_img))
    print('shape of ground_truth', np.shape(ground_truth))

    test_img_norm=test_img[:,:,0][:,:,None]
    print('shape of test_img_norm', np.shape(test_img_norm))

    test_img_input=np.expand_dims(test_img_norm, 0)
    print('shape of test_img_input', np.shape(test_img_input))

    #prediction = (model.predict(test_img_input))
    #prediction = (model.predict(test_img))

    #x = test_img
    #s=128
    #x = x.reshape(1,s,s,3)
    #x = x.reshape(s,s,3)

    prediction = model.predict(test_img_input)

    print('shape of prediction', np.shape(prediction))

    #print(prediction[:,60])

    predicted_img=np.argmax(prediction, axis=3)[0,:,:]
    print('shape of predicted_img', np.shape(predicted_img))

    print(predicted_img[:,80])



    plt.figure(figsize=(12, 8))
    plt.subplot(231)
    plt.title('Testing Image')
    plt.imshow(test_img[:,:,0], cmap='jet')
    
    plt.subplot(232)
    plt.title('Testing Label')
    plt.imshow(ground_truth[:,:,0], cmap='jet')
    plt.subplot(233)
    plt.title('Prediction on test image')
    plt.imshow(predicted_img, cmap='jet')
    plt.show()